In [2]:
import pandas as pd
import numpy as np

In [3]:
def get_data_with_xlwings(file_name, sheet):
    data = pd.read_excel(file_name, sheet_name=sheet)
    return data

In [4]:
def load_all_pricing_matrices():
    service_types = [
        "FHD Ground",
        "FHD Ground Canada",
        "FedEx First Overnight",
        "FedEx Priority Overnight",
        "FedEx Standard Overnight",
        "FedEx 2Day A.M.",
        "FedEx 2Day",
        "FedEx Express Saver",
        "Intra-Hawaii Standard List Rate"
    ]
    
    mw_service_types = [
#         ADDRESS STRING FORMATTING AT SOME POINT
        "MW FedEx First Overnight",
        "MW FedEx Priority Overnight",
        "MW FedEx Standard Overnight",
        "MW FedEx 2Day A.M.",
        "MW FedEx 2Day",
        "MW FedEx Express Saver",
        "MWIntra-HawaiiFedExPriorityON"
    ]
    
    freight_service_types = [
        "FedEx First Overnight Freight",
        "FedEx 1Day Freight",
        "FedEx 2Day Freight",
        "FedEx 3Day Freight"
    ]
    
    pricing_matrices = {}
    all_sheets = pd.read_excel("./separated_us_express_rates.xlsx", sheet_name=None)
    
    for service_type in service_types:
        pricing_matrices[service_type] = all_sheets[service_type]
        
    for service_type in mw_service_types:
        pricing_matrices[service_type] = all_sheets[service_type]
        
    for service_type in freight_service_types:
        pricing_matrices[service_type] = all_sheets[service_type]

    return pricing_matrices

In [5]:
def dim_check(l, w, h, dim):
    if np.isnan(l) or np.isnan(w) or np.isnan(h) or np.isnan(dim):
#         IF ANY VALUE IS NOT A NUMBER (nan), CANNOT RUN MATH OPS
        return None
    elif dim == 0:
#         ANY NUM/0 == UNDEFINED
        return None
    else:    
        return l*w*h/dim

In [19]:
def add_accessorials(row):
    total = float()
    
    if np.isnan(row['OFSC$']):
        pass
    else:
        total = total + row['OFSC$']
        
    if np.isnan(row['Late Fee_LF1-1']):
        pass
    else:
        total = total + row['Late Fee_LF1-1']
        
    if np.isnan(row['OTP$']):
        pass
    else:
        total = total + row['OTP$']
        
    if np.isnan(row['ODS$']):
        pass
    else:
        total = total + row['ODS$']
    
    if np.isnan(row['Adult Signature']):
        pass
    else:
        total = total + row['Adult Signature']
    
    if np.isnan(row['Print Return Label']):
        pass
    else:
        total = total + row['Print Return Label']
    
    if np.isnan(row['ORES$']):
        pass
    else:
        total = total + row['ORES$']
    
    if np.isnan(row['ODASC$']):
        pass
    else:
        total = total + row['ODASC$']
    
    if np.isnan(row['ODASEC$']):
        pass
    else:
        total = total + row['ODASEC$']
    
    if np.isnan(row['ODAS$']):
        pass
    else:
        total = total +row['ODAS$']
    
    if np.isnan(row['ODASER$']):
        pass
    else:
        total = total + row['ODASER$']
    
    if np.isnan(row['Address Correction']):
        pass
    else:
        total = total + row['Address Correction']
    
    if np.isnan(row['Return On Call Surcharge']):
        pass
    else:
        total = total + row['Return On Call Surcharge']
    
    if np.isnan(row['ONDOCAC$']):
        pass
    else:
        total = total + row['ONDOCAC$']
    
    return total

In [29]:
def process_data(data):
#     ENSURE HANDLING EXISTS FOR FedEx® Pak
#     MAY NEED TO SEPARATE PRICING FOR ZONES LISTED AS 9-10 AND 11-12 FOR SIMPLICITY OF CALC.

    pricing_matrices = load_all_pricing_matrices()
    
    for index, row in data.iterrows():
        weight = row['RW']
        zone = int(row['Z'])
        service_type = row['Service Type']
        otpub = row['OTPUB']
#         fuel_surcharge = row['Fuel Surcharge']
        svc_pkging = row['Service Packaging']
        l = row['L']
        w = row['W']
        h = row['H']
        dim = row['DIM']
        this_dim = dim_check(l, w, h, dim)
        
#         FEDEX STANDARD OVERNIGHT AND FEDEX 2DAY AM HAVE NO DATA FOR ZONES 10-12
#         FEDEX EXPRESS SAVER HAS NO DATA IN ZONE COLUMNS 9-10/11-12/13-16
        if svc_pkging == "FedEx Pak":
            pass
#             print("NO RATES PROVIDED TO CALCULATE PRICE")
#             otpub.color = (255, 0, 0)
        elif service_type in pricing_matrices:
#             GRAB CORRESPONDING DATA
            pricing_matrix_data = pricing_matrices[service_type]
#             DETERMINE THE LARGER WEIGHT FOR CALC
            actual_weight = weight if this_dim == None or weight > this_dim else this_dim
#             IF ITS AN ENVELOPE
            if actual_weight == 0.5:
                new_otpub = pricing_matrix_data.iloc[0, zone-1] 
                otpub = float("{:.2f}".format(new_otpub + add_accessorials(row)))
#                 print(otpub)
#             IF ITS NOT AN ENVELOPE
            else:
                new_otpub = pricing_matrix_data.iloc[int(actual_weight)+1, zone-1] 
                otpub = float("{:.2f}".format(new_otpub + add_accessorials(row)))

        elif service_type == "Ground":
            if row["Recipient Country/Territory"] != "US":
                pricing_matrix_data = pricing_matrices["FHD Ground Canada"]
                actual_weight = weight if this_dim == None or weight > this_dim else this_dim
                zone_index = int()
                
                if zone == 51:
                    zone_index = 1
                elif zone == 54:
                    zone_index = 2
                new_otpub = pricing_matrix_data.iloc[int(actual_weight)-1, zone_index]
                otpub = float("{:.2f}".format(new_otpub = add_accessorials(row)))
            else:
                pricing_matrix_data = pricing_matrices["FHD Ground"]
                actual_weight = weight if this_dim == None or weight > this_dim else this_dim
                zone_index = zone
                
                if zone == 14:
                    zone_index = 10
                elif zone == 17:
                    zone_index = 11
                elif zone == 22:
                    zone_index = 12
                elif zone == 23:
                    zone_index = 13
                elif zone == 25:
                    zone_index = 14
                elif zone == 92:
                    zone_index = 15
                elif zone == 96:
                    zone_index = 16
                    
                new_otpub = pricing_matrix_data.iloc[int(actual_weight)-1, zone_index-1]
                otpub = float("{:.2f}".format(new_otpub + add_accessorials(row)))

        else:
#             BELOW PRINTS ANY SERVICE TYPE FOR WHICH WE MAY HAVE NOT HAVE ANY HANDLING 
            print(f'{service_type} not addressed!')
#             BELOW CHANGES CELL TO COLOR RED TO EASILY LOCATE UNADRESSED CELLS
            otpub.color = (255, 0, 0)
    
#     AT THIS POINT, THE DATA MANIPULATED IS SIMPLY AN INSTANCE AND LATER WE MUST USE save_results_with_xlwings() FOR
#     MANIPULATED DATA TO PERSIST IN A WORKBOOK
    return data

In [30]:
def save_results_with_xlwings(data, file_name, sheet_name):
    print(data)
#     wb = xw.Book(file_name)
#     ws = wb.sheets[sheet_name]
#     ws.range('A1').options(index=False).value = data
#     wb.save()
#     wb.close()

In [31]:
#BEGIN MAIN SCRIPT USING ABOVE FUNCTIONS
file_name = './charges.xlsm'
spreadsheet_name = 'FREIGHT & ACCESSORIALS'
charge_data = get_data_with_xlwings(file_name, spreadsheet_name)
processed_data = process_data(charge_data)
# save_results_with_xlwings(processed_data, file_name, spreadsheet_name)